In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import emlearn

In [2]:
# Load datasets
train = pd.read_csv("dataset/train_radiation_data.csv")
test  = pd.read_csv("dataset/test_radiation_data.csv")

# Preprocessing function
def preprocess(df):
    """
    Preprocess the dataset with temporal parsing and encoding
    """
    df['time'] = pd.to_datetime(df['time'])
    df['hour'] = df['time'].dt.hour
    df['minute']  = df['time'].dt.minute
    df['weekofyr'] = df['time'].dt.isocalendar().week

    # Temporal encoding using sine and cosine
    df['hour_sin']  = np.sin(2 * np.pi * df['hour']/24)
    df['hour_cos']  = np.cos(2 * np.pi * df['hour']/24)
    df['min_sin']   = np.sin(2 * np.pi * df['minute']/60)
    df['min_cos']   = np.cos(2 * np.pi * df['minute']/60)
    df['woy_sin']   = np.sin(2 * np.pi * (df['weekofyr']-1)/52)
    df['woy_cos']   = np.cos(2 * np.pi * (df['weekofyr']-1)/52)
    return df

# Apply preprocessing
train = preprocess(train)
test  = preprocess(test)

# Features and Target
features = [
    'Gb(i)','Gd(i)','Gr(i)','H_sun','T2m','WS10m',
    'hour_sin','hour_cos','min_sin','min_cos','woy_sin','woy_cos'
]
target = ['P']

# Scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(train[features])
y_train_scaled = scaler_y.fit_transform(train[target])

X_test_scaled  = scaler_X.transform(test[features])
y_test_scaled  = scaler_y.transform(test[target])

# Sequence generation
LOOK_BACK = 16
HORIZON = 16

def create_sequences(X, y, look_back=LOOK_BACK, horizon=HORIZON):
    """
    Create sequences for time series prediction
    """
    Xs, ys = [], []
    for i in range(len(X) - look_back - horizon + 1):
        Xs.append(X[i:i + look_back])
        ys.append(y[i + look_back : i + look_back + horizon].flatten())
    return np.array(Xs), np.array(ys)

X_train, y_train = create_sequences(X_train_scaled, y_train_scaled)
X_test, y_test   = create_sequences(X_test_scaled, y_test_scaled)

# Flatten input for feed-forward model
n_features = len(features)
X_train_flat = X_train.reshape(X_train.shape[0], LOOK_BACK * n_features)
X_test_flat  = X_test.reshape(X_test.shape[0], LOOK_BACK * n_features)

# Build Feed-Forward Model
def build_ffnn_model(input_shape, horizon):
    """
    Modello feed-forward compatibile con emlearn
    """
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(input_shape,)))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(horizon, activation='linear'))

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Create and summarize model
model = build_ffnn_model(LOOK_BACK * n_features, HORIZON)
model.summary()

# Training
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    X_train_flat, y_train,
    epochs=20,
    batch_size=32,
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)


print(f"Final validation loss: {min(history.history['val_loss']):.6f}")
print(f"Final validation MAE: {min(history.history['val_mae']):.6f}")

# Evaluation
loss, mae = model.evaluate(X_test_flat, y_test)
print(f"\n\nTest Loss: {loss:.4f}, Test MAE: {mae:.4f}")

c:\Users\andre\anaconda3\envs\DMML\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,960 (58.44 KB)

 Trainable params: 14,960 (58.44 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
4382/4382 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.0105 - mae: 0.0573 - val_loss: 0.0083 - val_mae: 0.0508
Epoch 2/20
4382/4382 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0076 - mae: 0.0462 - val_loss: 0.0088 - val_mae: 0.0532
Epoch 3/20
4382/4382 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0071 - mae: 0.0438 - val_loss: 0.0093 - val_mae: 0.0538
Epoch 4/20
4382/4382 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0068 - mae: 0.0423 - val_loss: 0.0091 - val_mae: 0.0535
Epoch 5/20
4382/4382 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0067 - mae: 0.0416 - val_loss: 0.0081 - val_mae: 0.0501
Epoch 6/20
4382/4382 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0066 - mae: 0.0410 - val_loss: 0.0095 - val_mae: 0.0548
Epoch 7/20
4382/4382 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - loss: 0.0065 - mae: 0.0405 - val_loss: 0.0079 - val_mae: 0.0488
Epoch 8/20
4382/4382 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - loss: 0.0064 - mae: 0.0402 - val_loss: 0.0080 - val_mae: 0.0480
Epoch 9/20
4382/4382 ━━━━━━━━━━━

In [4]:
print("Converting model for IoT deployment...")

cmodel = emlearn.convert(model, method='inline')
cmodel.save(file="../sensorPV/modelTiny.h", name='modelTiny')

Converting model for IoT deployment...


'\n#include <eml_net.h>\nstatic const float modelTiny_layer_0_biases[64] = { -0.018755f, 0.114608f, -0.020346f, -0.110074f, 0.000000f, -0.101755f, -0.014537f, 0.095947f, -0.021714f, -0.082559f, -0.026136f, -0.005927f, -0.152230f, -0.010946f, -0.011007f, -0.006003f, -0.013289f, -0.019915f, 0.143874f, -0.027530f, -0.020769f, 0.168622f, -0.021852f, -0.009618f, -0.167699f, -0.013117f, -0.021757f, -0.008692f, -0.027523f, -0.020269f, -0.012741f, -0.020135f, -0.007811f, -0.021630f, -0.007102f, -0.020152f, -0.098577f, -0.008283f, 0.000000f, -0.015829f, -0.037755f, -0.028218f, -0.013741f, -0.025879f, -0.024464f, -0.033468f, 0.126426f, -0.018159f, -0.133805f, -0.023467f, 0.000000f, -0.005992f, -0.017206f, -0.027593f, -0.010215f, -0.013760f, 0.205986f, -0.000223f, -0.110159f, -0.016060f, -0.136634f, -0.032034f, -0.183353f, -0.047945f };\nstatic const float modelTiny_layer_0_weights[12288] = { -0.000969f, 0.264432f, 0.027488f, 0.045578f, -0.014108f, 0.009046f, -0.135336f, 0.333675f, 0.021679f, -0.